In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import RegexpTokenizer, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from string import punctuation
import collections
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#import en_core_web_smfrom sklearn.feature_extraction.text import CountVectorizer, TfidfTransformerfrom sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score

In [ ]:
import numpy as np 
import pandas as pd 
import re

# Data Visualization
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

import missingno as msno


import seaborn as sns
sns.set(style='whitegrid')

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
tweets_data_path = '/content/Corona_NLP_train.csv'
train = pd.read_csv(tweets_data_path, encoding = 'latin-1')
test = pd.read_csv('/content/Corona_NLP_test.csv')

In [ ]:
len(train)

In [ ]:
train.isna().sum()

In [ ]:
train['UserName'].nunique()

In [ ]:
train['ScreenName'].nunique()

In [ ]:
train.head()

In [ ]:
def remove_urls(text):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
train['Content']=train['OriginalTweet'].apply(lambda x:remove_urls(x))

In [ ]:
def remove_urls(text):
    return re.sub(r'<.*?>', '', text)
train['Content']=train['Content'].apply(lambda x:remove_urls(x))

In [ ]:
train.head()

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt

In [ ]:
train['pfinal'] = np.vectorize(remove_pattern)(train['Content'], '@[\w]*')

In [ ]:
train.head()

In [ ]:
import re
train['final'] = train['pfinal'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [ ]:
train.head()

In [ ]:
train['final'] = train['final'].str.replace('[^a-zA-Z#]+',' ')

In [ ]:
train['final'] = train['final'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))
train.head()

In [ ]:
tokenized_tweet = train['final'].apply(lambda x: x.split())

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

# apply stemmer for tokenized_tweet
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])

In [ ]:
train.head(2)

In [ ]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

In [ ]:
train['final'] = tokenized_tweet

In [ ]:
train.head()

In [ ]:
new_df = train[['final','Sentiment']]
new_df.head()

In [ ]:
train['Sentiment'].value_counts()

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
new_df['final'].apply(lambda x: [item for item in x if item not in stop])

In [ ]:
new_df.head(6)

In [ ]:
new_df.isnull().sum()

In [ ]:
new_df.final[1]

In [ ]:
from sklearn.model_selection import train_test_split

train,valid = train_test_split(new_df,test_size = 0.2,random_state=0,stratify = new_df.Sentiment.values) 
print("train shape : ", train.shape)
print("valid shape : ", valid.shape)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
vectorizer = CountVectorizer(decode_error = 'replace',stop_words = stop)

X_train = vectorizer.fit_transform(train.final.values)
X_valid = vectorizer.transform(valid.final.values)

y_train = train.Sentiment.values
y_valid = valid.Sentiment.values

print("X_train.shape : ", X_train.shape)
print("X_train.shape : ", X_valid.shape)
print("y_train.shape : ", y_train.shape)
print("y_valid.shape : ", y_valid.shape)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

naiveByes_clf = MultinomialNB()

naiveByes_clf.fit(X_train,y_train)

NB_prediction = naiveByes_clf.predict(X_valid)
NB_accuracy = accuracy_score(y_valid,NB_prediction)
print("training accuracy Score    : ",naiveByes_clf.score(X_train,y_train))
print("Validation accuracy Score : ",NB_accuracy )
print(classification_report(NB_prediction,y_valid))


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()

rf_clf.fit(X_train,y_train)

rf_prediction = rf_clf.predict(X_valid)
rf_accuracy = accuracy_score(y_valid,rf_prediction)
print("Training accuracy Score    : ",rf_clf.score(X_train,y_train))
print("Validation accuracy Score : ",rf_accuracy )
print(classification_report(rf_prediction,y_valid))

In [ ]:
from sklearn.svm import SVC

svc = SVC()

svc.fit(X_train, y_train)

svc_prediction = svc.predict(X_valid)
svc_accuracy = accuracy_score(y_valid,svc_prediction)
print("Training accuracy Score    : ",svc.score(X_train,y_train))
print("Validation accuracy Score : ",svc_accuracy )
print(classification_report(svc_prediction,y_valid))